This file contains the code for cleaning and analysis of the group case study on investment in course 2 of the PGDDS.

## Code Style
    - Case: 
        - snake_case for objects
        - camelCase for functions and classes
    - Double quotes first, then single quotes

## Libraries used
    - Pandas
    - Numpy

## Obejctives of Analysis
Identify the most heavily invested main sectors in each of the three countries (for funding type FT and investments range of 5-15 M USD).

Business objective: Identify the best: a. Sectors; b. Countries; c. Investment rounds for Spark Funds.

## Metric
Mean amount of money invested in a particular country. 

## The Workflow
The workflow for this analysis is rather simple. Focus on answering the questions asked in the checkpoints. Following this flow, the code in this .ipynb is organized according to the checkpoints. There will be a clear heading indicating the starting and ending of each checkpoint and question.

In [ ]:
# importing dependencies
# numpy
import numpy as np # version: 1.15.0

# pandas
import pandas as pd # version: 0.23.4

# Checkpoint 1: Data Cleaning
There are five tasks in this checkpoint:
    - Number of unique companies in rounds2.csv
	- Number of unique companies in companies.tsv
	- Key column from the companies dataset that can be used to merge it with rounds data
	- Organizations in companies that are missing in rounds2.
    - Merge the two datasets.

## Subtask 1.1: Importing the data
The first step of the analysis is to import the two main datasets that we will be needing for the analysis: companies and rounds. 

In [ ]:
# import companies.csv as companies
companies = pd.read_csv("companies.txt", sep = "\t", encoding = "ISO-8859-1") 

# import rounds2.csv as rounds
rounds2 = pd.read_csv("rounds2.csv", sep = ",", encoding = "ISO-8859-1")
# ISO for lack of charset in UTF-8

In [ ]:
#information of the companies dataset
print(companies.info()); print("shape of dataset: ", companies.shape); print("variable dtypes:\n", companies.dtypes)

In [ ]:
# information about the rounds dataset
print(rounds2.info()); print("shape of dataset: ", rounds2.shape); print("variable dtypes:\n", rounds2.dtypes)

## Subtask 1.2: Cleaning Data

Since we will be focusing mostly on four variables only, let's remove all the extraneous variables from both the datasets. 
We'll remove from the companies dataset the following variables:
    - state_code
    - region
    - city
    - homepage_url
    - founded_at
    - name

In [ ]:
# removing unnecessary columns from companies
companies.drop(["state_code", "region", "city", "homepage_url", "founded_at", "name"], axis = 1, inplace = True)

We'll remove the following from rounds:
    - funded_at
    - funding_round_code
    - funding_round_permalink

In [ ]:
# removing unnecessary columns from rounds
rounds2.drop(["funded_at", "funding_round_code", "funding_round_permalink"], axis = 1, inplace = True)

## Checkpoint 1 Q1: Number of unique companies in rounds
To do this, we'll use the company_permalink column. However, instead of doing this directly, we'll first convert the company_permalink to lowercase and then determine the number of unique records.

In [ ]:
# converting company_permalink to lower case and getting number of unique records.
rounds2.company_permalink.str.lower().nunique()

There seem to be 66370 unique companies in the dataset. This means that there are companies that had more than one round of funding.

## Checpoint 1 Q2: Number of unique companies in companies
This time, we'll use the permalink, which is supposed to be the UID of a company. As with rounds.company_permalink, we'll first convert to lower case and then proceed to count the number of unique records.

In [ ]:
companies.permalink.str.lower().nunique()

The companies dataset has 66368 unique companies. There seems to be a discrepancy between the number of unique records in companies and rounds. Does this mean that there are at least 2 companies in rounds that are not present in companies?

## Checkpoint 1 Q3: Key column to merge companies and rounds
From the data dictionary we know the companies.permalink and rounds.company_permalink are UID's of each company in the dataset. So, we'll use companies.permalink as the key to merge with rounds.

## Checkpoint 1 Q4: Mismatches between rounds and companies
Ok. Now, we're required to find out if there are any records that are unique to rounds only. That is these organizations are not present in companies but are present in rounds.

We can do this by merging on companies.permalink and rounds.company_permalink. But, we'll take a slightly different approach here. 

Instead, the same result can be achieved by checking if every permalink in rounds.company_permalink is present in company.permalink. 

In [ ]:
# converting rounds.company_permalink and companies.permalink to lower case
companies["permalink"] = companies.permalink.str.lower()

rounds2["company_permalink"] = rounds2.company_permalink.str.lower()

In [ ]:
# checking if there are any unique records.
print(rounds2.company_permalink[~rounds2.company_permalink.isin(companies.permalink)].dropna());

There seem to be 7 companies that are in rounds but not in companies. So, the answer to the fourth question is yes. There are organizations that are present in rounds but not in companies.

## Checkpoint 1 Q5: Merge the two dataframes
This is the basis of all our analysis. Merging the two DataFrames will give us a single data frame which contains all the data needed. Let's get the cursory look.

In [ ]:
pd.merge(rounds2, companies, how = "inner", left_on = "company_permalink", 
                  right_on = "permalink")

### Drop Unnecessary rows

So, after some thought it was decided unanimously that the best approach would be to drop all missing values from the raised_amount_usd column. This decision was made because imputing values in the target column might lead to biased results. 

In [ ]:
# dropping all missing values from the rounds DataFrame.
rounds2.dropna(inplace = True)

In [ ]:
rounds2.shape # new shape of rounds: (94959, 3)

Dropping those missing values leaves us with 95K observations, which is not a problem as we still left with > 75% of the observations. The next step is to combine the two datasets.

### Merge the rounds2 and companies data

In [ ]:
master_frame = pd.merge(rounds2, companies, how = "inner", left_on = "company_permalink", 
                  right_on = "permalink")

In [ ]:
# checking for missing values
master_frame.isnull().mean().sort_values(ascending = False)

There are two variables with missing values: country_code (an important variable) has 6% values missing and category_list has 1% values missing. 

The next thing to do is to check for duplicates. If there are duplicates, just drop them.

In [ ]:
# checking for duplicates in the master DataFrame
master_frame.duplicated().sum() # there seem to be 1311 duplicated values.

In [ ]:
master_frame.drop_duplicates(keep = "first", inplace = True)

So, dropping duplicates resulted in the loss of about 2K rows. We're now left with about 93K rows. This is about 81% of the initial observations. Deleting any more values is a good idea. This would result in the loss of more data, bringing the total down even more. 

### Treating the remaining missing values
Since the remaining missing values are present in series of dtype objects. This means that instead of just deleting the values, we can flag them, there by retaining values in the entire dataset.

In [ ]:
# replacing missing values in country_code with new sentinel value
master_frame["country_code"].fillna("unknown", inplace = True)

In [ ]:
# on running the following code, we see that the missing value percentage has gone down
master_frame.isnull().mean().sort_values(ascending = False)

Now, we have about 1% of the total values missing 1% of it's data. 1% of 93647 is about 940. We can go ahead and delete these rows. We'll still be left with 80.5% of the total observations.

In [ ]:
master_frame.dropna(inplace = True) # Now that we have this DataFrame, we can finally start with the analysis.

In [ ]:
# checking the information about the master table after performing all the operations
print(master_frame.info()); print(master_frame.shape)

# Checkpoint 2: Funding Type Analysis

## Subtask 2.1 Investment Analysis
### Change the unit of 'raised_amount_usd' column

The investment is in terms of 'million USD', convert the unit of the `raised_amount_usd` from `$` to `million $`.

Converting raised_amount_usd from dollars to millions of dollars would increase the readability of the results.

In [ ]:
# code for unit conversion here
master_frame["raised_amount_usd"] = master_frame["raised_amount_usd"] / 1000000

### Calculate the average investment amount for each of the four funding types (venture, angel, seed, and private equity) 

In [ ]:
master_frame.groupby("funding_round_type").raised_amount_usd.mean().sort_values()

## Checkpoint 1 Q: 
### Table 2.1: Average Values of Investments for Each of these Funding Types 
 Average funding amount of venture type: 11.682595 (million USD) 	                              
 Average funding amount of angel type: 0.961039 (million USD)	 
 Average funding amount of seed type: 0.721313 (million USD)	 
 Average funding amount of private equity type: 73.350449 (million USD)	 

Considering that Spark Funds wants to invest between 5 to 15 million USD per investment round, which investment type is the most suitable for it?

venture
	                                                                                       

# Checkpoint 3: Country Analysis

 ## Subtask 3.1: Find the top nine countries with highest total funding for the given investment = "venture"

    -- Spark Funds wants to see the top nine countries which have received the highest total funding (across ALL sectors for the chosen investment type)

    -- For the chosen investment type, make a data frame named top9 with the top nine countries (based on the total investment amount each country has received)

In [ ]:
#groupby command here to find the country-wise total funding for the investment type 
master_grpby_country= master_frame.loc[master_frame.funding_round_type == "venture", :].groupby("country_code")

In [ ]:
#code to find the top nine countries with highest total funding
top9 = master_grpby_country["raised_amount_usd"].sum().sort_values(ascending=False)[:9]
print(top9)

## Subtask 3.2: Identify the top three English-speaking countries in the data frame top9.

### Table 3.1: Analysing the Top 3 English-Speaking Countries
Based on the list of countries where English is an official language - the top three English-speaking countries are:

 1. Top English-speaking country:    USA (United States)             
 2. Second English-speaking country: GBR (United Kingdom)
 3. Third English-speaking country:  IND (India)

# Checkpoint 4: Sector Analysis 1
So, let's start off with the sector analysis. Here's what needs to be done in this part of the analysis.
    - Extract the primary sector of each category list from the category_list column.
    - Use the mapping file 'mapping.csv' to map each primary sector to one of the eight main sectors (Note that ‘Others’ is also considered one of the main sectors)

The primary sector is the string that appears before the first pipe "|" in the category_list variable. So, let's get on with it.

## Subtask 4.1 Extracting the primary sector from category_list
The first step is to create a new variable called primary_sector to store the primary sector of each organization in the dataframe.

The primary sector is the first string that results from splitting master.category_list on "|".

In [ ]:
# creating the primary sector
master_frame["primary_sector"] = master_frame.category_list.str.split("|").str.get(0)

The next step is to map each primary sector in master.primary_sector to a main sector. This is in the mapping.csv file.

## Subtask 4.2 Map each primary sector to one of the eight main sectors 

In [ ]:
# loading mapping.csv as mapping
mapping = pd.read_csv("mapping.csv", sep = ",", encoding = "ISO-8859-1")

In [ ]:
# some basic information about mapping
print(mapping.info())

In [ ]:
mapping.head()

In [ ]:
# dropping the Blanks column since this just serves as a flag to identify NaN's
mapping.drop("Blanks", axis = 1, inplace = True)

# dropping the single NaN at the head of the dataset
mapping.dropna(axis = 0, inplace = True)

Here are the final steps to finish up checkpoint 4.
1. Merge master and mapping_tidy
2. Check for: a. Missing values; b. Duplicates
3. Treat missing values and drop duplicates`

This mapping file has turned out to be a one-to-one sparse matrix that maps each primary sector to one of the eight main sectors. 
The main task here is this: convert the wide (and sparse) representation of the mapping into a long representation. This means bringing in all the colums under one roof.

Before this is done though, a check to see if every there are any records present in master.primary_sector are not in mapping.category_list.

In [ ]:
# primary sectors present in master.primary_sector but not present in mapping_tidy.category_list
master_frame.primary_sector[~master_frame.primary_sector.isin(mapping.category_list)].dropna().unique()

From the result of the code above, it's clear that there are 89 sub-sectors that are not included in the mapping dataset. To deal with this issue, the sectors have been manually assigned a main_sector by observation.`

These are primary sectors that are present in master.primary_sector but not in mapping.category_list.
    - Analytics 
    - Finance 
    - Financial Services 
    - Finance Technology 
    - Business Analytics 
    - Big Data Analytics 
    - Investment Management 
    - Social Media Advertising 
    - Personal Finance 
    - Predictive Analytics 
    - Financial Exchanges
    - Mobile Analytics 
    - Social Media Management
    - Promotional
    - Waste Management
    - Natural Gas Uses
    - Biotechnology and Semiconductor
    - Green Tech
    - Energy Management
    - Natural Resources
    - Alternative Medicine
    - Cannabis
    - Medical Professionals
    - Personal Health
    - Mobile Emergency&Health

On closer inspection of the mapping dataframe, it was seen that there are some misspelt words in mapping.category_list. And, most of them are just misspelt words in the list written above. This will be the approach to fix the problem:
1. Get an idea of the words that are misspelt and their number
2. Replace them with their correct spellings in the mapping dataframe
3. Merge with the master dataframe
4. Treat any missing values that might arise because of the merge.

In [ ]:
# categories in mapping but not in master.primary_sector
print(mapping.category_list[~mapping.category_list.isin(master_frame.primary_sector)].dropna().unique());
print(mapping.category_list[~mapping.category_list.isin(master_frame.primary_sector)].dropna().unique().size)

There are about 61 words that need to be corrected. Instead of correcting each mis-spelling separately, it is more efficient to write a function to do that. Then, a for loop and a list of words can be used to correct each spelling mistake. 

In [ ]:
# function to replace missplet words
def wordReplacer(wrong_spelling, correct_spelling):
    """ This function takes in two parameters: the wrong_spelling and the correct_spelling.
    It returns mapping.category_list with the modified spellings.
    """
    return mapping["category_list"].str.replace(pat = wrong_spelling, repl = correct_spelling, regex = False)

In [ ]:
# list of words that need to be replaced
list_of_words = [["Alter0tive", "Alternative"], ["A0lytics", "Analytics"], ["Ma0gement", "Management"], 
                 ["Can0bis", "Cannabis"], ["Fi0nce", "Finance"], ["Sig0ge", "Signage"], 
                 ["Fi0ncial", "Financial"], ["Gover0nce", "Governance"], ["Jour0lism", "Journalism"], 
                 ["Professio0ls", "Professionals"], ["0notechnology", "Nanotechnology"], 
                 ["0tural", "Natural"], ["0vigation", "Navigation"], ["Perso0l", "Personal"],
                 ["Perso0lization", "Personalization"], ["Professio0l", "Professional"], 
                 ["Promotio0l", "Promotional"], ["Veteri0ry", "Veterinary"], ["Chi0", "China"], 
                 ["Educatio0l", "Educational"]]

In [ ]:
# replacing the misspelt words with the correct spelling
for i in list_of_words:
    mapping["category_list"] = wordReplacer(i[0], i[1])

In [ ]:
# checking for missing categories again.
master_frame.primary_sector[~master_frame.primary_sector.isin(mapping.category_list)].dropna().unique()

There still are some primary_sectors missing. This can be solved by creating bins for each of the missing sectors and assigning them to a main sector manually.

In [ ]:
# creating lists of primary_sectors that fall under a main sector

# Cleantech / Semiconductors
cleantech_semiconductors = ["Natural Gas Uses", "Biotechnology and Semiconductor", "GreenTech"]

# Health
health = ["Mobile Emergency&Health", "Psychology"]

# Entertainment
entertainment = ["Internet TV", "Skill Gaming", "Racing"]


In [ ]:
# gathering all the columns under one roof
mapping_long = pd.melt(mapping, id_vars = ["category_list"], var_name = "main_sector", value_name = "yes_no")

# tidying mapping_long to produce the final version of the mapping dataset
mapping_tidy = mapping_long.loc[mapping_long.yes_no == 1, ["category_list", "main_sector"]]

To finish up, let's take one look at the mapping_tidy dataset just to make sure everything is alright

In [ ]:
print(mapping_tidy.info());
print(mapping_tidy.isnull().mean()) # no null values. We can proceed to merge the two datasets!

## Subtask 4.3: Generate the merged data frame with each primary sector mapped to its main sector

These will be the final steps to get the merged dataframe:
1. Merge the mapping_tidy dataset with the master dataset using an left join
2. Impute the missing main_sector values in the merged master dataset
3. Replace the other missing values in main_sector with the "Others" flag.

In [ ]:
# merging the master and mapping_tidy
master_frame = pd.merge(master_frame, mapping_tidy, how = "left", left_on = "primary_sector", right_on = "category_list")

In [ ]:
# removing the category_list_y variable and renaming category_list_x as category_list
master_frame.drop("category_list_y", axis = 1, inplace = True)

master_frame.rename(index = str, columns = {"category_list_x": "category_list"}, inplace = True)

In [ ]:
# filling up the main_sector with appropriate values.
# social, analytics, finance, advertising
master_frame.loc[master_frame.primary_sector.isin(["Social Media Advertising"]), "main_sector"] = \
                                                    "Social, Finance, Analytics, Advertising"

# cleantech / semiconductors
master_frame.loc[master_frame.primary_sector.isin(cleantech_semiconductors), "main_sector"] = "Cleantech / Semiconductors"

# health
master_frame.loc[master_frame.primary_sector.isin(health), "main_sector"] = "Health"

# filling up the remaining missing values with others
master_frame.loc[master_frame.main_sector.isnull(), "main_sector"] = "Others"

Now, the dataset can be used to for the analysis.

### Export this dataframe to work in Tableau for Checkpoint 6

In [ ]:
master_frame.to_csv('master_file.csv', sep=',')

# Checkpoint 5: Sector Analysis 2

The aim is to find out the most heavily invested main sectors in each of the three countries (for funding type FT and investments range of 5-15 M USD).

The steps to obtain the deliverables of this checkpoint:
1. Create a dataframe for each country with the preferred funding type.
2. Add the total investements and counts of investements in each sector to the dataframes.
3. Fill out the table with the results we get.

## Subtask 5.1: Create a dataframe for each country with the preferred funding type

In [ ]:
# creating the three dataframes
# usa
D1 = master_frame.loc[(master_frame.country_code == "USA") & (master_frame.funding_round_type == "venture"), :]

# great britain
D2 = master_frame.loc[(master_frame.country_code == "GBR") & (master_frame.funding_round_type == "venture"), :]

# india
D3 = master_frame.loc[(master_frame.country_code == "IND") & (master_frame.funding_round_type == "venture"), :]

## Subtask 5.2: Add the total investements and counts of investements in each sector to the dataframes.

In [ ]:
# getting the total investements and adding them to the dataframes
#usa
D1_summary = D1.groupby("main_sector").raised_amount_usd.agg(["sum", "count"])

D1 = pd.merge(D1, D1_summary, how = "left", on = "main_sector")

# great britain
D2_summary = D2.groupby("main_sector").raised_amount_usd.agg(["sum", "count"])

D2 = pd.merge(D2, D2_summary, how = "left", on = "main_sector")

# india
D3_summary = D3.groupby("main_sector").raised_amount_usd.agg(["sum", "count"])

D3 = pd.merge(D3, D3_summary, how = "left", on = "main_sector")

Now that we have the dataframes, it's time to answer the questions asked.

## Subtask 5.3: Fill out the table with the results.
### Table 5.1 : Sector-wise Investment Analysis

In [ ]:
# total number of investments in countries
# usa
print("Number of investments made in USA:", D1.shape[0])

# great britain
print("Number of investements made in Great Britain:", D2.shape[0])

# india
print("Number of investments made in India:", D3.shape[0])

 1. Total number of investments (count) in each country:
USA: 35292
GBR: 2027
IND: 813

In [ ]:
# total size of investments in countries
top3 = master_frame.loc[(master_frame.country_code.isin(["USA", "GBR", "IND"])) & (master_frame.funding_round_type == "venture"), :]
top3.groupby("country_code").raised_amount_usd.agg(["sum", "count"])

2. Total amount of investment (USD) in each country:
USA: 411102.768986(million USD)
GBR: 19931.867246(million USD)
IND: 14134.008718(million USD)

In [ ]:
# top three sectors based on count of investments
# usa
print("USA", "\n", D1.groupby("main_sector").raised_amount_usd.count().sort_values(ascending = False), "\n")

# great britain
print("Great Britain", "\n", D2.groupby("main_sector").raised_amount_usd.count().sort_values(ascending = False), 
      "\n")

# india
print("India", "\n", D3.groupby("main_sector").raised_amount_usd.count().sort_values(ascending = False), "\n")

3. Top sector (based on count of investments):
USA: Others
GBR: Others
IND: Others

4. Second-best sector (based on count of investments):
USA: Cleantech / Semiconductors
GBR: Cleantech / Semiconductors
IND: Social, Finance, Analytics, Advertising

5. Third-best sector (based on count of investments):
USA: Social, Finance, Analytics, Advertising
GBR: Social, Finance, Analytics, Advertising
IND: News, Search and Messaging

6. Number of investments in the top sector (refer to point 3):
USA:8521
GBR:526
IND:285

7. Number of investments in the second-best sector (refer to point 4):
USA:7723
GBR:436
IND:144

8. Number of investments in the third-best sector (refer to point 5):
USA:6984
GBR:414
IND:139

In [ ]:
# For the top sector count-wise (point 3), which company received the highest investment?
# usa
print("USA", 
      D1[D1.main_sector == "Others"].groupby("permalink").raised_amount_usd.sum().sort_values(ascending = False)[:10],
     "\n")

# gbr
print("Great Britain", 
      D2[D2.main_sector == "Others"].groupby("permalink").raised_amount_usd.sum().sort_values(ascending = False)[:10],
     "\n")

# ind
print("India", 
      D3[D3.main_sector == "Others"].groupby("permalink").raised_amount_usd.sum().sort_values(ascending = False)[:10],
     "\n")

9. For the top sector count-wise (point 3), which company received the highest investment?
USA: social-finance 
GBR: oneweb
IND: flipkart

In [ ]:
# For the second-best sector which company received the highest investment?
# usa
print("USA", "\n",
      D1[D1.main_sector == "Cleantech / Semiconductors"].groupby("permalink").raised_amount_usd.sum().sort_values(ascending = False)[:10],
     "\n")

# gbr
print("Great Britain", "\n",
      D2[D2.main_sector == "Cleantech / Semiconductors"].groupby("permalink").raised_amount_usd.sum().sort_values(ascending = False)[:10],
     "\n")

# ind
print("India", "\n",
      D3[D3.main_sector == "Social, Finance, Analytics, Advertising"].groupby("permalink").raised_amount_usd.sum().sort_values(ascending = False)[:10],
     "\n")

10. For the second-best sector count-wise (point 4), which company received the highest investment?
USA: Freescale
GBR: ImmunoCore
IND: Shopclues.com